In [11]:
import pandas as pd
from elasticsearch import Elasticsearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
import boto3
from elasticsearch import helpers

from sklearn.neighbors import NearestNeighbors
import pickle
# Based on the ata
# data = pd.read_csv('Cleaned_data.csv')
# artist_info = data[['artist','title']]
# emotions = data[['anger','fear','joy','sadness','surprise']]

closest_songs_model = NearestNeighbors(n_neighbors=5)
closest_songs_model.fit(emotions)
pickle.dump(closest_songs_model, open('closest_songs_model.sav', 'wb'))
_,indecies=closest_songs_model.kneighbors([[0.17561023, 0.12503634, 0.16991208, 0.49357194, 0.03326787]])
[dict(artist_info.iloc[row]) for  row in range(len(indecies[0]))]




[{'artist': 'Talking Heads', 'title': 'Artists Only'},
 {'artist': 'Blake Shelton', 'title': 'Neon Light'},
 {'artist': 'Keith Urban', 'title': 'All for You'},
 {'artist': 'Charlie Puth', 'title': 'How Long'},
 {'artist': 'The Hit Crew', 'title': 'Hamster Dance'}]

In [2]:

# result = loaded_model.score(X_test, Y_test)
# print(result)

FileNotFoundError: [Errno 2] No such file or directory: 'closest_songs_model.sa'

In [12]:

host = 'search-gifnote-kyl54r7f5kd3ehsewzr4i73elu.us-east-1.es.amazonaws.com' 
region = 'us-east-1' # e.g. us-west-1
service = 'es'
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service)
es = Elasticsearch(
    hosts = [{'host': host, 'port': 443}],
    http_auth = awsauth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)


In [13]:
try:
    loaded_model = pickle.load(open('closest_songs_model.sa', 'rb'))
except FileNotFoundError:
    results_dictionary = es.search(index='songs',size=100)['hits']['hits']
    data = pd.DataFrame([song['_source']['doc'] for song in results_dictionary])
    artist_info = data[['artist','title']]
    emotions = data[['anger','fear','joy','sadness','surprise']]
    closest_songs_model = NearestNeighbors(n_neighbors=5)
    closest_songs_model.fit(emotions)
    _,indecies=closest_songs_model.kneighbors([[0.17561023, 0.12503634, 0.16991208, 0.49357194, 0.03326787]])
    [dict(artist_info.iloc[row]) for  row in range(len(indecies[0]))]
    

'last night i fell asleep and i saw you dancing in my dreams just like the autumn leaves have fall for you all for you  you changed my life you changed my ways i dont even recognize myself these days it must be a reflection of you only you  i cant remember feeling love like this in so long i cant imagine living life without you by my side day after day you find a way to make this grown man cry its so true im all for you  my arms still hold you tight keep you warm in the coldest night my hand to pull your hair right back away from your eyes for you only you  if i knew how to do it id paint the moon the stars around you paint the perfect kind of sunset but i couldnt make it more beautiful than you oh nothing compares to you you know its true  i cant remember feeling love like this in so long i cant imagine living life without you by my side day after day you find a way to make this grown man cry its so true im all for you im all for you  i cant remember feeling love like this in so long 